In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


######################################################
# Music Stuff
######################################################
from dbBase import dbBase
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData




import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-11-16 20:41:54.668243


## Parse Artists Base

In [2]:
%load_ext autoreload
%autoreload
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData
maindb = mainDB(mdb=None, create=True, debug=False)
pdb    = parseDBArtistsData(maindb=maindb)
dbdata = maindb.dbdata

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================


## Create Metadata (Only)

In [ ]:
dbs    = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
dbs    = ['AllMusic']
start,cmt=clock("Parsing Metadata")
for db in dbs:
    pdb.parseMetadata(db=db, nProcs=3, modVals=list(range(100)), force=True) 
elapsed(start, cmt)

## Parse Everything

In [ ]:
ff   864K Nov 15 00:41 7-DB.p
-rwxrwxrwx    1 tgadfort  staff   953K Nov 15 00:41 0-DB.p
-rwxrwxrwx    1 tgadfort  staff   1.0M Nov 15 00:43 8-DB.p
-rwxrwxrwx    1 tgadfort  staff   884K Nov 15 00:43 1-DB.p
-rwxrwxrwx    1 tgadfort  staff   1.0M Nov 15 00:44 2-DB.p
-rwxrwxrwx    1 tgadfort  staff   932K Nov 15 00:50 6-DB.p
-rwxrwxrwx    1 tgadfort  staff   963K Nov 15 00:51 42-DB.p
-rwxrwxrwx    1 tgadfort  staff   1.0M Nov 15 00:53 43-DB.p
-rwxrwxrwx    1 tgadfort  staff   996K Nov 15 00:54 44-DB.p
-rwxrwxrwx    1 tgadfort  staff   963K Nov 15 00:55 38-DB.p
-rwxrwxrwx    1 tgadfort  staff   873K Nov 15 00:57 39-DB.p
-rwxrwxrwx    1 tgadfort  staff   978K Nov 15 00:58 40-DB.p
-rwxrwxrwx    1 tgadfort  staff   845K Nov 15 01:00 41-DB.p
    
    
    
-rwxrwxrwx    1 tgadfort  staff   873K Nov 15 00:57 39-DB.p
-rwxrwxrwx    1 tgadfort  staff   1.0M Nov 15 00:43 8-DB.p
-rwxrwxrwx    1 tgadfort  staff   864K Nov 15 00:41 7-DB.p

    
    -rwxrwxrwx    1 tgadfort  staff   956K Nov 16 13:30 88-DB.p
-rwxrwxrwx    1 tgadfort  staff   1.1M Nov 16 13:30 98-DB.p
-rwxrwxrwx    1 tgadfort  staff   947K Nov 16 13:30 89-DB.p
-rwxrwxrwx    1 tgadfort  staff   359K Nov 16 20:23 97-DB.p
-rwxrwxrwx    1 tgadfort  staff   355K Nov 16 20:23 87-DB.p
-rwxrwxrwx    1 tgadfort  staff   912K Nov 16 20:36 37-DB.p


In [5]:
dbs    = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
#dbs = ['MusicBrainz', 'LastFM']
dbs = ["Discogs"]
#dbs = ["Discogs", "MusicBrainz"]
#dbs = ["AllMusic"]
#dbs = ["MusicBrainz"]
#dbs = []
start,cmt=clock("Parsing Everything")
for db in dbs:
    s = 10
    #pdb.parse(db, nProcs=3, modVals=list(range(s,s+9)), force=False)
    #pdb.parse(db, nProcs=4, modVals=list(range(10,100)), force=True)
    pdb.parse(db, nProcs=3, modVals=list(range(100)), force=False, extra=False)
    #pdb.parse(db, nProcs=3, modVals=[88,89,87,37,98,97], force=True, extra=False)
    #pdb.parse(db, nProcs=3, modVals=[3,4,5], force=False)    
elapsed(start, cmt)

Current Time is Mon Nov 16, 2020 20:49:10 for Parsing Everything
Parsing Discogs with 3 processes using [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]] mod values.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Parsing Artist Files For ModVal 0Parsing Artist Files For ModVal 9Parsing Artist Files For ModVal 18


  ===> Found 7532 files, but there are no new files to parse so skipping.
-----------

Saving 7318 new artist IDs media data to /Users/tgadfort/Music/Discog/artists-discogs-db/metadata/45-MediaMetadata.p
  --> This file is 4.1MB.
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 46
  ===> Found 7427 files, but there are no new files to parse so skipping.
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 40
  ===> Found 7400 files, but there are no new files to parse so skipping.
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 47
  ===> Found 7500 files, but there are no new files to parse so skipping.
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 31
  ===> Found 7305 files, but there are no new files to parse so skipping.
-----------------

----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 61
  --> This file is 4.5MB.
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 75
  ===> Found 7438 files, but there are no new files to parse so skipping.
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 71
  ===> Found 7521 files, but there are no new files to parse so skipping.
----------------------------------------------------------------------------------------------------  ===> Found 7506 files, but there are no new files to parse so skipping.

Parsing Artist Files For ModVal 76
----------------------------------------------------------------------------------------------------
Parsing Artist Files For ModVal 62
  ===> Found 7481 files, but there are no new files to parse so skipping

In [ ]:
data = getFile("/Users/tgadfort/Music/Discog/artists-musicbrainz-db/13-DB.p")

In [ ]:
data["134347080492261077195869941664563052313"]

In [ ]:
dbdata["MusicBrainz"]["Artists"].parseArtistModValExtraFiles(13)

In [ ]:
for i in range(71,100):
    dbdata["MusicBrainz"]["Artists"].assertDBModValExtraData(i, test=False)

In [ ]:
dbdata["MusicBrainz"]["Artists"].getData("/Users/tgadfort/discogs/artists-musicbrainz/13/134347080492261077195869941664563052313.p").show()

In [ ]:
dbdata["MusicBrainz"]["Artists"].getData("/Users/tgadfort/discogs/artists-musicbrainz/13/extra/134347080492261077195869941664563052313-2.p").show()

In [ ]:
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
print("\n\n{0}\n".format("Full DB"))
maindb.setDBFull("AllMusic")
#maindb.setDBFull("MusicBrainz")
#maindb.setDBFull("Discogs")
#maindb.setDBFull("LastFM")
#maindb.setDBFull()
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

elapsed(start, cmt)

## Parse Single ModVal

In [ ]:
for i in range(10):
    for j in range(10):
        print("{0}touch /Volumes/Piggy/Discog/artists-discogs/{1}/0*0*0*".format("!", i*10+j))
        print("{0}echo '{1}'".format("!", i+10*j))

In [ ]:
start,cmt=clock("Parsing Single ModVal")
pdb.parse(db="Discogs", modVals=[0], nProcs=1, force=False)
elapsed(start, cmt)

In [ ]:
data = getFile('/Users/tgadfort/Music/Discog/artists-discogs-db/0-DB.p')

In [ ]:
len(data)

In [ ]:
def getArtistNumAlbums(artistData):
    numAlbums = sum([len(x) for x in artistData.media.media.values()])
    return numAlbums
saveIt = True
dbNumAlbums = sum([getArtistNumAlbums(artistData) for artistData in data.values()])
print('0', dbNumAlbums)


In [ ]:
data['11792020000']

In [ ]:
dbdata["LastFM"]["Artists"].parseArtistModValExtraFiles(0, force=True, debug=True)

In [ ]:
from glob import glob
glob('/Volumes/Piggy/Discog/artists-lastfm/0/extra/11792020000-*.p')

In [ ]:
bsdata = getHTML(ifile)
ols = bsdata.findAll("ol", {"class": "buffer-standard"})
ols

In [ ]:
ifile='/Volumes/Piggy/Discog/artists-lastfm/0/extra/72742897500-2.p'
#getFile(ifile)
a       = dbdata["LastFM"]["Artist"].getData(ifile, debug=False)

def info(a):
    name    = a.artist.name
    ID      = a.ID.ID
    url     = a.url.url
    pages   = a.pages
    media   = a.media
    counts  = a.mediaCounts
    profile = a.profile

    print("{0: <10}{1}".format("Name:", name))
    print("{0: <10}{1}".format("ID:", ID))
    print("{0: <10}{1}".format("URL:", url))
    print("{0: <10}{1}".format("Pages:", pages.get()))
    print("{0: <10}{1}".format("Profile:", profile.get()))
    print("{0: <10}{1}".format("Counts:", counts.get()))
    print("{0: <10}".format("Media:"))
    for mediaType,mediaData in media.get()['media'].items():
        for mediaInfo in mediaData:
            print("\t{0: <10}{1}".format(mediaType, mediaInfo.get()['album']))

In [ ]:
info(a)

In [ ]:
nProcs = 8
%load_ext autoreload
%autoreload
from mainDB import mainDB
maindb = mainDB(mdb=None, create=True, debug=True)
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")

parseArtistsParallel(db="AllMusic", nProcs=nProcs)
maindb.setDBFull("AllMusic")

parseArtistsParallel(db="MusicBrainz", nProcs=nProcs)
maindb.setDBFull("MusicBrainz")

parseArtistsParallel(db="Discogs", nProcs=nProcs)
maindb.setDBFull("Discogs")

#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
parseArtistsParallel(db="LastFM", nProcs=nProcs)
maindb.setDBFull("LastFM")

#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Master DB

In [ ]:

print("\n\n{0}\n".format("Full DB"))
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

# Download My Known Artists

In [ ]:
mdb = myMusicDBMap(debug=True)

In [ ]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [ ]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 3404)

In [ ]:
# Refused

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['MusicBrainz']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)